# TP2 - Sistemas Autónomos
#### "Monitorização de parâmetros ambientais para a prática de desporto"

## Import Libraries

In [ ]:
import sys
import config
import firebase_admin
from firebase_admin import auth, credentials, firestore
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt

## Get the Data


### Get the Database from Cloud Firestore

In [ ]:
# Get the key
cred = credentials.Certificate(config.firestore_key)

# Initialize/Get the app
try:
    app = firebase_admin.initialize_app(cred)
except:
    app = firebase_admin.get_app()

# Get the database
db = firestore.client()

### Get the 'WM', 'UV' and 'AQ' collections

In [ ]:
# ------- Open Weather Map -------

dataWM = pd.DataFrame(columns=['year','month','day','hours','minutes','hours.minutes','temperature','general_weather','humidity','pressure','temp_min','temp_max','wind_speed'])

wm_ref = db.collection(u'WM')
docsWM = wm_ref.stream()

# general_weather tem que se meter em integer, mas temos que decidir se continuamos a extrair o "description" ou o "main" no momento do request

for doc in docsWM:
    params = doc.to_dict()
    datetime = re.split(r'[-T:]',doc.id)
    perc_minutes = int(datetime[3])+(0.0167*int(datetime[4])) # precisava de ter os minutos adaptados para a unidade da hora
    dataWM = dataWM.append({'year':int(datetime[0]),'month':int(datetime[1]) ,'day':int(datetime[2]), 'hours':int(datetime[3]), 'minutes': int(datetime[4]), 'hours.minutes': perc_minutes, 'temperature': float(params['feels_like'])-273.15, 'general_weather':params['general_weather'], 'humidity': int(params['humidity']), 'pressure': int(params['pressure']), 'temp_min': float(params['temp_min'])-273.15, 'temp_max': float(params['temp_max'])-273.15, 'wind_speed': float(params['wind_speed'])}, ignore_index=True)


# ------- Open UV -------

dataUV = pd.DataFrame(columns=['uv','uv_time','uv_max','uv_max_time','st1','st2','st3','st4','st5','st6'])

uv_ref = db.collection(u'UV')
docsUV = uv_ref.stream()

for doc in docsUV:
    params = doc.to_dict()
    dataUV = dataUV.append({'uv': float(params['uv']), 'uv_time': params['uv_time'], 'uv_max': float(params['uv_max']), 'uv_max_time': params['uv_max_time'], 'st1': params['st1'], 'st2': params['st2'], 'st3': params['st3'], 'st4': params['st4'], 'st5': params['st5'], 'st6': params['st6']}, ignore_index=True)


# ------- Open AQ -------

dataAQ = pd.DataFrame(columns=['no2','o3','pm10'])

aq_ref = db.collection(u'AQ')
docsAQ = aq_ref.stream()

for doc in docsAQ:
    params = doc.to_dict()
    dataAQ = dataAQ.append({'no2': int(params['no2']), 'o3': int(params['o3']), 'pm10': int(params['pm10'])}, ignore_index=True)


data = pd.concat([dataWM, dataUV, dataAQ], axis=1)
data.head()


In [ ]:
data.info()

## Exploratory Data Analysis

In [ ]:
sns.set_palette("GnBu_d")
sns.set_style('whitegrid')

In [ ]:

sns.jointplot(x='hours.minutes', y='temperature', data=data)


In [ ]:
sns.jointplot(x='hours.minutes', y='uv', data=data)


In [ ]:
sns.jointplot(x='hours.minutes', y='o3', data=data)